# SETUP

## Import Libraries

In [23]:
from pymongo import MongoClient
import pandas as pd
import numpy as np

## Read MongoDB

In [24]:
client = MongoClient('mongodb://localhost:27017')
db = client["test"]

## Initialize Output DataFrame

In [25]:
output_df = pd.DataFrame()

# MATCHAPPS

## Matchapps Initalization

### Create Matchapps DataFrame

In [26]:
matchapps_collection = db["matchapps"]
matchapps_df = matchapps_collection.find()

### Flatten, reorganize, and clean Data

In [27]:
matchapps_df = pd.json_normalize(matchapps_df)

matchapps_column_order = [
    "_id",
    "metadata.matchNumber",
    "metadata.scouterName",
    "metadata.robotTeam",
    "metadata.robotPosition",
    "leftStartingZone",
    "climb",
    "trapNotes",
    "autoNotes.near",
    "autoNotes.mid",
    "autoNotes.far",
    "autoNotes.amp",
    "autoNotes.miss",
    "teleNotes.near",
    "teleNotes.mid",
    "teleNotes.far",
    "teleNotes.amp",
    "teleNotes.miss"   
]

matchapps_df = matchapps_df[matchapps_column_order]
matchapps_df.sort_values("metadata.matchNumber", inplace=True)
matchapps_df

,_id,metadata.matchNumber,metadata.scouterName,metadata.robotTeam,metadata.robotPosition,leftStartingZone,climb,trapNotes,autoNotes.near,autoNotes.mid,autoNotes.far,autoNotes.amp,autoNotes.miss,teleNotes.near,teleNotes.mid,teleNotes.far,teleNotes.amp,teleNotes.miss
0,65f294e1a3ae8f61fff75ce1,1,Jim,3177,red_1,True,none,1,4,0,1,3,0,5,8,1,2,6
1,65f294e1a3ae8f61fff75ce5,1,Jim,3177,red_2,False,park,0,0,2,3,1,2,3,6,9,4,5
2,65f294e1a3ae8f61fff75ce9,1,Jim,8411,red_3,False,park,1,2,4,4,2,3,9,7,8,6,7
3,65f294e1a3ae8f61fff75ced,1,Jim,4341,blue_1,False,none,0,1,4,0,2,4,5,2,5,5,1
4,65f294e1a3ae8f61fff75cf1,1,Jim,8274,blue_2,True,none,0,1,2,3,1,1,8,0,2,3,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2390,65f294eba3ae8f61fff78239,399,Jim,4019,red_3,True,center,0,3,1,2,0,0,9,6,6,4,3
2392,65f294eba3ae8f61fff78241,399,Jim,1728,blue_2,True,source,1,4,0,3,2,1,7,8,6,3,0
2388,65f294eba3ae8f61fff78231,399,Jim,545,red_1,False,amp,0,1,3,1,1,2,7,9,2,5,9
2389,65f294eba3ae8f61fff78235,399,Jim,9759,red_2,True,park,1,1,1,3,4,2,4,3,1,2,0


## Matchapps Function and Variable Initializations

### Check Alliance Harmonization Function Definition

In [28]:
def check_team_harmonization_in_alliance(t1, t2, t3):
    t1_status, t2_status, t3_status = False, False, False
    if t1 not in ["failed", "none", "park"]:
        if t1 == t2:
            t1_status = True
            t2_status = True
        if t1 == t3:
            t1_status = True
            t3_status = True
    elif t2 not in ["failed", "none", "park"]:
        if t2 == t3:
            t2_status = True
            t3_status = True
    return t1_status, t2_status, t3_status

### Complete List of all Teams in DF

In [29]:
team_list = matchapps_df["metadata.robotTeam"].unique().tolist()

### Complete Range of All Matches

In [30]:
match_range = matchapps_df["metadata.matchNumber"].max()

## Matchapps Data Analysis

### Auto Data Analysis Calculations

#### Auto Speaker Notes Average (near, mid, far)

In [31]:
output_df["Auto Speaker Notes Near Avg"] = matchapps_df.groupby("metadata.robotTeam")["autoNotes.near"].mean()
output_df["Auto Speaker Notes Mid Avg"] = matchapps_df.groupby("metadata.robotTeam")["autoNotes.mid"].mean()
output_df["Auto Speaker Notes Far Avg"] = matchapps_df.groupby("metadata.robotTeam")["autoNotes.far"].mean()

output_df["Auto Speaker Notes Total Avg"] = None

for team in team_list:
    output_df.at[team, "Auto Speaker Notes Total Avg"] = sum([output_df.at[team, "Auto Speaker Notes Near Avg"], output_df.at[team, "Auto Speaker Notes Mid Avg"], output_df.at[team, "Auto Speaker Notes Far Avg"]])

In [32]:

grouped = matchapps_df.groupby("metadata.robotTeam")
output_df["Auto Speaker Notes Near Max"] = grouped["autoNotes.near"].max()
output_df["Auto Speaker Notes Mid Max"] = grouped["autoNotes.mid"].max()
output_df["Auto Speaker Notes Far Max"] = grouped["autoNotes.far"].max()
output_df["Auto Speaker Notes Total Max"] = (matchapps_df[["autoNotes.far", "autoNotes.mid", "autoNotes.near"]].sum(axis=1)).groupby(matchapps_df['metadata.robotTeam']).max()

output_df

,Auto Speaker Notes Near Avg,Auto Speaker Notes Mid Avg,Auto Speaker Notes Far Avg,Auto Speaker Notes Total Avg,Auto Speaker Notes Near Max,Auto Speaker Notes Mid Max,Auto Speaker Notes Far Max,Auto Speaker Notes Total Max
metadata.robotTeam,,,,,,,,
116,0.736842,2.157895,2.052632,4.947368,4,4,4,10
155,1.947368,2.315789,1.947368,6.210526,4,4,4,11
175,2.291667,2.000000,1.708333,6.0,4,4,4,12
545,2.173913,2.043478,2.000000,6.217391,4,4,4,11
596,1.960000,2.120000,2.560000,6.64,4,4,4,12
...,...,...,...,...,...,...,...,...
9492,2.320000,2.640000,1.680000,6.64,4,4,4,11
9640,1.645161,2.129032,1.935484,5.709677,4,4,4,11
9759,1.944444,2.722222,2.500000,7.166667,4,4,4,12


#### Auto Amp Notes Average

In [33]:
output_df["Auto Notes Amp Avg"] = matchapps_df.groupby("metadata.robotTeam")["autoNotes.amp"].mean()

#### Auto Percent of Miss Notes

In [34]:
output_df["Auto Percent of Missed Notes"] = None

 # CHANGE FORMAT LATER
output_df["Auto Max Missed Notes"] = None
for team in team_list:
    missed_notes = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.miss"].sum()
    total_notes = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.near"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.mid"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.far"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.amp"].sum()
    + missed_notes
    output_df.at[team, "Auto Percent of Missed Notes"] = missed_notes / total_notes
    
     # CHANGE FORMAT LATER

    output_df.at[team, "Auto Max Missed Notes"] = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.amp"].max()

#### Auto All Notes Average

In [35]:
output_df["Auto Notes Avg"] = (output_df["Auto Notes Amp Avg"] +
                               output_df["Auto Speaker Notes Near Avg"] +
                               output_df["Auto Speaker Notes Mid Avg"] +
                               output_df["Auto Speaker Notes Far Avg"]) / 4

#### Average Auto points

In [36]:
output_df['Auto Points Avg'] = (5 * (matchapps_df['teleNotes.near'] + matchapps_df['teleNotes.mid'] + matchapps_df['teleNotes.far']) + 2 * matchapps_df['teleNotes.amp']).groupby(matchapps_df['metadata.robotTeam']).mean()

### Teleop Data Analysis Calculations

#### Teleop Speaker Notes Average (near, mid, far)

In [37]:
output_df["Tele Speaker Notes Near Avg"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.near"].mean()
output_df["Tele Speaker Notes Mid Avg"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.mid"].mean()
output_df["Tele Speaker Notes Far Avg"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.far"].mean()

output_df["Tele Speaker Notes Total Avg"] = None

for team in team_list:
    output_df.at[team, "Tele Speaker Notes Total Avg"] = sum([output_df.at[team, "Tele Speaker Notes Near Avg"], output_df.at[team, "Tele Speaker Notes Mid Avg"], output_df.at[team, "Tele Speaker Notes Far Avg"]])

In [38]:
output_df["Tele Speaker Notes Near Max"] = grouped["teleNotes.near"].max()
output_df["Tele Speaker Notes Mid Max"] = grouped["teleNotes.mid"].max()
output_df["Tele Speaker Notes Far Max"] = grouped["teleNotes.far"].max()
output_df["Tele Speaker Notes Total Max"] = (matchapps_df[["teleNotes.far", "teleNotes.mid", "teleNotes.near"]].sum(axis=1)).groupby(matchapps_df['metadata.robotTeam']).max()

output_df

,Auto Speaker Notes Near Avg,Auto Speaker Notes Mid Avg,Auto Speaker Notes Far Avg,Auto Speaker Notes Total Avg,Auto Speaker Notes Near Max,Auto Speaker Notes Mid Max,Auto Speaker Notes Far Max,Auto Speaker Notes Total Max,Auto Notes Amp Avg,Auto Percent of Missed Notes,...,Auto Notes Avg,Auto Points Avg,Tele Speaker Notes Near Avg,Tele Speaker Notes Mid Avg,Tele Speaker Notes Far Avg,Tele Speaker Notes Total Avg,Tele Speaker Notes Near Max,Tele Speaker Notes Mid Max,Tele Speaker Notes Far Max,Tele Speaker Notes Total Max
metadata.robotTeam,,,,,,,,,,,,,,,,,,,,,
116,0.736842,2.157895,2.052632,4.947368,4,4,4,10,2.157895,2.785714,...,1.776316,73.000000,3.210526,5.473684,4.315789,13.0,9,9,9,22
155,1.947368,2.315789,1.947368,6.210526,4,4,4,11,1.894737,1.243243,...,2.026316,82.263158,5.105263,5.105263,4.157895,14.368421,9,9,9,24
175,2.291667,2.000000,1.708333,6.0,4,4,4,12,1.875000,0.981818,...,1.968750,75.583333,4.833333,4.375000,4.291667,13.5,9,9,9,21
545,2.173913,2.043478,2.000000,6.217391,4,4,4,11,2.260870,0.84,...,2.119565,72.217391,4.913043,4.391304,3.173913,12.478261,9,9,8,26
596,1.960000,2.120000,2.560000,6.64,4,4,4,12,1.880000,0.979592,...,2.130000,80.000000,3.840000,5.080000,5.560000,14.48,9,9,8,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9492,2.320000,2.640000,1.680000,6.64,4,4,4,11,1.720000,0.931034,...,2.090000,77.680000,4.680000,4.560000,4.680000,13.92,9,9,9,21
9640,1.645161,2.129032,1.935484,5.709677,4,4,4,11,1.903226,1.45098,...,1.903226,77.774194,4.354839,4.129032,5.290323,13.774194,9,9,9,23
9759,1.944444,2.722222,2.500000,7.166667,4,4,4,12,1.777778,0.914286,...,2.236111,70.833333,4.333333,3.555556,4.388889,12.277778,9,9,8,20


#### Teleop Amp Notes Average

In [39]:
output_df["Tele Notes Amp Avg"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.amp"].mean()

 # CHANGE FORMAT LATER
output_df["Tele Max Amp Notes"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.amp"].max()

#### Teleop All Notes Average

In [40]:
output_df["Tele Notes Avg"] = (output_df["Tele Notes Amp Avg"] +
                               output_df["Tele Speaker Notes Near Avg"] +
                               output_df["Tele Speaker Notes Mid Avg"] +
                               output_df["Tele Speaker Notes Far Avg"]) / 4

 # CHANGE FORMAT LATER
output_df["Tele Max Notes"] = None

#### Teleop Percent of Missed Notes

In [41]:
output_df["Tele Percent of Missed Notes"] = None

 # CHANGE FORMAT LATER
output_df["Tele Max Missed Notes"] = None
for team in team_list:
    missed_notes = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.miss"].sum()
    total_notes = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.near"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.mid"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.far"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.amp"].sum()
    + missed_notes
    output_df.at[team, "Percent of Missed Notes"] = missed_notes / total_notes
    
     # CHANGE FORMAT LATER

    output_df.at[team, "Tele Max Missed Notes"] = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.amp"].max()

In [42]:
output_df["Tele Speaker Notes Near Max"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.near"].max()
output_df["Tele Speaker Notes Mid Max"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.mid"].max()
output_df["Tele Speaker Notes Far Max"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.far"].max()

#### Average Teleop Points

In [43]:
output_df['Tele Points Avg'] = (2 * (matchapps_df['teleNotes.near'] + matchapps_df['teleNotes.mid'] + matchapps_df['teleNotes.far']) + 1 * matchapps_df['teleNotes.amp']).groupby(matchapps_df['metadata.robotTeam']).mean()

In [44]:
# output_df['Endgame Points Avg'] =
#(np.where(matchapps_df['trapNotes']), 0, 1)
np.where(matchapps_df['climb'].isin(['source', 'center', 'amp']), 0, 3)


array([3, 3, 3, ..., 0, 3, 3])

### Other Data Analysis Calculations

#### Trap Notes

In [45]:
output_df["Trap Notes Avg"] = matchapps_df.groupby("metadata.robotTeam")["trapNotes"].mean()

#### Percent of Successful Climbs and Successful Climbs Total

In [46]:
output_df["Percent of Successful Climbs"] = None
output_df["Successful Climbs Total"] = None

for team in team_list:
    climb_list = matchapps_df.loc[matchapps_df["metadata.robotTeam"] == team, "climb"].tolist()
    total_climbs = 0
    successful_climbs = 0
    for climb_type in climb_list:
        if climb_type == "failed":
            total_climbs += 1
        elif climb_type == "amp" or climb_type == "source" or climb_type == "center":
            total_climbs += 1
            successful_climbs += 1
    if total_climbs == 0:
        output_df.at[team, "Percent of Successful Climbs"] = 0
        
    else:
        output_df.at[team, "Percent of Successful Climbs"] = successful_climbs / total_climbs
    
    output_df.at[team, "Successful Climbs Total"] = successful_climbs

#### Harmonized Climbs Total

#### Percent of Harmonized Climbs out of Successful Climbs

In [47]:
# output_df["Percent of Harmonized Climbs out of Successful Climbs"] = 0

# output_df.index

# for team in team_list:
#     output_df.at[team, "Percent of Harmonized Climbs out of Successful Climbs"] = output_df.at[team, "Harmonized Climbs Total"] / output_df.at[team, "Successful Climbs Total"]

#### Percent of Times Robot Left Starting Zone During Autos

In [48]:
output_df["Percent of Times Robot Left Starting Zone During Autos"] = matchapps_df.groupby("metadata.robotTeam")["leftStartingZone"].mean()

### Custom Metric Data Analysis Calculations

#### Endgame Custom Metric

In [49]:
# output_df["Endgame Custom Metric"] = ((output_df["Percent of Successful Climbs"] * 5) ** 2 + (output_df["Percent of Harmonized Climbs out of Successful Climbs"] * 5) ** 2 + (output_df["Trap Notes Avg"] * 5) ** 2) / 3
# output_df["Endgame Custom Metric"] = output_df["Endgame Custom Metric"].apply(lambda x: round(x, 1))

# SUPERAPPS

## Supperapps Initalization

### Create Superapps DataFrame

In [50]:
superapps_collection = db["superapps"]
superapps_df = superapps_collection.find()

### Flatten, reorganize, and clean Data

In [51]:
superapps_df = pd.json_normalize(superapps_df)
superapps_df.sort_values("metadata.matchNumber", inplace=True)
superapps_df

,_id,defense,defended,comments,__v,metadata.scouterName,metadata.matchNumber,metadata.robotTeam,metadata.robotPosition,fouls.inBot,...,fouls.tipEntangBot,fouls.pinBot,fouls.podiumFoul,fouls.zoneFoul,fouls.stageFoul,fouls.overExtChute,fouls.multiplePieces,humanShooter.highNotes.amp,humanShooter.highNotes.source,humanShooter.highNotes.center
3,65f294e1a3ae8f61fff75ce3,someDef,False,"[great_driving, good_driving, mid_defense, stu...",0,Jim,1,3177,red_1,1,...,1,1,1,1,0,0,0,False,False,True
4,65f294e1a3ae8f61fff75ce7,noDef,True,"[clogging, avoids_under_stage]",0,Jim,1,3177,red_2,0,...,0,0,1,1,1,1,0,NaN,NaN,NaN
5,65f294e1a3ae8f61fff75ceb,fullDef,False,"[great_driving, mid_defense, sturdy_build, avo...",0,Jim,1,8411,red_3,0,...,0,1,1,1,1,0,0,NaN,NaN,NaN
6,65f294e1a3ae8f61fff75cef,fullDef,True,"[good_driving, source_only, ineffective_defense]",0,Jim,1,4341,blue_1,0,...,0,0,0,1,0,1,1,NaN,NaN,NaN
7,65f294e1a3ae8f61fff75cf3,fullDef,True,"[sturdy_build, avoids_under_stage]",0,Jim,1,8274,blue_2,1,...,1,1,1,1,1,1,1,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2393,65f294eba3ae8f61fff7823b,fullDef,False,"[clogging, mid_defense, ineffective_defense, d...",0,Jim,399,4019,red_3,1,...,0,1,1,1,1,1,0,NaN,NaN,NaN
2395,65f294eba3ae8f61fff78243,noDef,False,[ineffective_defense],0,Jim,399,1728,blue_2,1,...,0,1,0,0,0,1,1,NaN,NaN,NaN
2391,65f294eba3ae8f61fff78233,someDef,True,"[good_driving, clogging]",0,Jim,399,545,red_1,0,...,1,1,1,1,1,0,0,NaN,NaN,NaN
2392,65f294eba3ae8f61fff78237,noDef,False,"[source_only, clogging, effective_defense, stu...",0,Jim,399,9759,red_2,0,...,0,0,0,1,0,1,0,True,True,False


## Superapps Data Analysis

### Average Fouls per Match for Different Foul Types

In [52]:
output_df["Average In Bot Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.inBot"].mean()
output_df["Average Damage Bot Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.damageBot"].mean()
output_df["Average Tip Entangle Bot Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.tipEntangBot"].mean()
output_df["Average Pin Bot Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.pinBot"].mean()
output_df["Average Podium Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.podiumFoul"].mean()
output_df["Average Zone Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.zoneFoul"].mean()
output_df["Average Stage Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.stageFoul"].mean()
output_df["Average Over Exc Chute Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.overExtChute"].mean()

### Average Total Fouls per Match

In [53]:
output_df["Average Fouls per Match"] = None

for team in team_list:
    output_df.at[team, "Average Fouls per Match"] = sum([
        output_df.at[team, "Average In Bot Fouls per Match"]
        + output_df.at[team, "Average Damage Bot Fouls per Match"]
        + output_df.at[team, "Average Tip Entangle Bot Fouls per Match"]
        + output_df.at[team, "Average Pin Bot Fouls per Match"]
        + output_df.at[team, "Average Podium Fouls per Match"]
        + output_df.at[team, "Average Zone Fouls per Match"]
        + output_df.at[team, "Average Stage Fouls per Match"]
        + output_df.at[team, "Average Over Exc Chute Fouls per Match"]])

### Percent of Different Defense Types in Matches AND Main Defense Type

In [54]:
output_df["Percent of Matches with No Defense"] = None
output_df["Percent of Matches with Some Defense"] = None
output_df["Percent of Matches with Full Defense"] = None
output_df["Main Defense Type"] = None

for team in team_list:
    
    defense_type_list = superapps_df.loc[superapps_df["metadata.robotTeam"] == team, "defense"].tolist()
    
    no_defense_count = 0
    some_defense_count = 0
    full_defense_count = 0
    
    for defense_type in defense_type_list:
        if defense_type == "noDef":
            no_defense_count += 1
        elif defense_type == "someDef":
            some_defense_count += 1
        elif defense_type == "fullDef":
            full_defense_count += 1

    defense_total_count = sum([no_defense_count, some_defense_count, full_defense_count])
        
    output_df.at[team, "Percent of Matches with No Defense"] = no_defense_count / defense_total_count
    output_df.at[team, "Percent of Matches with Some Defense"] = some_defense_count / defense_total_count
    output_df.at[team, "Percent of Matches with Full Defense"] = full_defense_count / defense_total_count
    
    biggest_defense_count = max([no_defense_count, some_defense_count, full_defense_count])
    main_defense_type = ""

    if biggest_defense_count == no_defense_count:
        main_defense_type += "No Defense "
    if biggest_defense_count == some_defense_count:
        main_defense_type += "Some Defense "
    if biggest_defense_count == full_defense_count:
        main_defense_type += "Full Defense"
    
    output_df.at[team, "Main Defense Type"] = main_defense_type

## Comment Counts

In [ ]:
for i in

## Additionals

# FINALIZE AND SEND DATA

## Send Data

In [55]:
output_df['teamNumber'] = output_df.index

In [56]:
output_df.to_json("../server/static/output_analysis.json", orient='records', default_handler=str)
output_df

,Auto Speaker Notes Near Avg,Auto Speaker Notes Mid Avg,Auto Speaker Notes Far Avg,Auto Speaker Notes Total Avg,Auto Speaker Notes Near Max,Auto Speaker Notes Mid Max,Auto Speaker Notes Far Max,Auto Speaker Notes Total Max,Auto Notes Amp Avg,Auto Percent of Missed Notes,...,Average Podium Fouls per Match,Average Zone Fouls per Match,Average Stage Fouls per Match,Average Over Exc Chute Fouls per Match,Average Fouls per Match,Percent of Matches with No Defense,Percent of Matches with Some Defense,Percent of Matches with Full Defense,Main Defense Type,teamNumber
metadata.robotTeam,,,,,,,,,,,,,,,,,,,,,
116,0.736842,2.157895,2.052632,4.947368,4,4,4,10,2.157895,2.785714,...,0.578947,0.526316,0.315789,0.631579,4.421053,0.263158,0.473684,0.263158,Some Defense,116
155,1.947368,2.315789,1.947368,6.210526,4,4,4,11,1.894737,1.243243,...,0.578947,0.473684,0.736842,0.421053,4.421053,0.263158,0.368421,0.368421,Some Defense Full Defense,155
175,2.291667,2.000000,1.708333,6.0,4,4,4,12,1.875000,0.981818,...,0.541667,0.666667,0.583333,0.375000,4.208333,0.25,0.291667,0.458333,Full Defense,175
545,2.173913,2.043478,2.000000,6.217391,4,4,4,11,2.260870,0.84,...,0.521739,0.478261,0.434783,0.652174,3.73913,0.304348,0.434783,0.26087,Some Defense,545
596,1.960000,2.120000,2.560000,6.64,4,4,4,12,1.880000,0.979592,...,0.400000,0.440000,0.600000,0.600000,3.84,0.24,0.52,0.24,Some Defense,596
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9492,2.320000,2.640000,1.680000,6.64,4,4,4,11,1.720000,0.931034,...,0.600000,0.360000,0.520000,0.520000,3.92,0.24,0.48,0.28,Some Defense,9492
9640,1.645161,2.129032,1.935484,5.709677,4,4,4,11,1.903226,1.45098,...,0.290323,0.419355,0.580645,0.516129,4.16129,0.419355,0.354839,0.225806,No Defense,9640
9759,1.944444,2.722222,2.500000,7.166667,4,4,4,12,1.777778,0.914286,...,0.611111,0.388889,0.611111,0.388889,3.777778,0.333333,0.444444,0.222222,Some Defense,9759


In [57]:
client.close()